In [ ]:
# Progetto Machine Learning: Riconoscimento di Specie di Uccelli con PASST

Questo notebook implementa un sistema di riconoscimento di specie di uccelli attraverso l'analisi di registrazioni audio della competizione BirdClef 2025. Il progetto utilizza l'architettura PASST (Patchout Audio Spectrogram Transformer) per classificare gli audio convertiti in spettrogrammi Mel e include anche un sistema di configurazione automatica dell'ambiente per eseguire il codice su Kaggle, Google Colab o in locale.

In [ ]:
# Librerie di sistema e utilità
import os
import sys
import platform
import time
import warnings
import logging
import datetime
from pathlib import Path
import pprint as pp
import seaborn as sns
from collections import Counter
import IPython.display as ipd
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Librerie per data science e manipolazione dati
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from sklearn.model_selection import train_test_split

# Librerie per elaborazione audio
import librosa
import librosa.display

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchaudio
import timm

# Per il modello PASST
import torchvision.transforms as transforms
from torch.nn import functional as F

# Visualizzazione
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# Ignoriamo i warning
warnings.filterwarnings("ignore")

# Configurazione del logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger('BirdClef-PASST')

print("Librerie importate con successo!")
print(f"PyTorch versione: {torch.__version__}")
print(f"timm versione: {timm.__version__}")
print(f"Python versione: {platform.python_version()}")
print(f"Sistema operativo: {platform.system()} {platform.release()}")

In [ ]:
# Pulizia directory di lavoro (utile per Kaggle)
import shutil
import os

# Imposta questo a True per abilitare la cancellazione
clear_working_dir = False  # Disabilitato di default per sicurezza

working_dir = '/kaggle/working/'

if clear_working_dir and os.path.exists(working_dir):
    for filename in os.listdir(working_dir):
        file_path = os.path.join(working_dir, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)  # elimina file o link
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)  # elimina directory
        except Exception as e:
            print(f'Errore durante la rimozione di {file_path}: {e}')
    print(f"Tutti i file in {working_dir} sono stati rimossi.")
else:
    print("Pulizia disabilitata (clear_working_dir = False)")

In [ ]:
# Variabile per impostare manualmente l'ambiente
# Modifica questa variabile in base all'ambiente in uso:
# - 'kaggle' per l'ambiente Kaggle
# - 'colab' per Google Colab
# - 'local' per l'esecuzione in locale
MANUAL_ENVIRONMENT = ''  # Impostare su 'kaggle', 'colab', o 'local' per forzare l'ambiente

def detect_environment():
    """
    Rileva se il notebook è in esecuzione su Kaggle, Google Colab o in locale.
    Rispetta l'impostazione manuale se fornita.
    
    Returns:
        str: 'kaggle', 'colab', o 'local'
    """
    # Se l'ambiente è stato impostato manualmente, usa quello
    if MANUAL_ENVIRONMENT in ['kaggle', 'colab', 'local']:
        print(f"Utilizzo ambiente impostato manualmente: {MANUAL_ENVIRONMENT}")
        return MANUAL_ENVIRONMENT
    
    # Verifica Kaggle con metodo più affidabile
    # Verifica l'esistenza di directory specifiche di Kaggle
    if os.path.exists('/kaggle/working') and os.path.exists('/kaggle/input'):
        print("Rilevato ambiente Kaggle")
        return 'kaggle'
    
    # Verifica se è Google Colab
    try:
        import google.colab
        return 'colab'
    except ImportError:
        pass
    
    # Se non è né Kaggle né Colab, allora è locale
    return 'local'

# Rileva l'ambiente attuale
ENVIRONMENT = detect_environment()
print(f"Ambiente rilevato: {ENVIRONMENT}")

In [ ]:
class Config:
    def __init__(self):
        # Rileva l'ambiente
        self.environment = ENVIRONMENT  # Usa la variabile globale impostata in precedenza
        
        # Imposta i percorsi di base in base all'ambiente
        if self.environment == 'kaggle':
            self.COMPETITION_NAME = "birdclef-2025"
            self.BASE_DIR = f"/kaggle/input/{self.COMPETITION_NAME}"
            self.OUTPUT_DIR = "/kaggle/working"
            self.MODELS_DIR = "/kaggle/input"  # Per i modelli pre-addestrati
            
            # Imposta subito i percorsi derivati per l'ambiente Kaggle
            self._setup_derived_paths()
            
        elif self.environment == 'colab':
            # In Colab, inizializza directory base temporanee
            self.COMPETITION_NAME = "birdclef-2025"
            self.OUTPUT_DIR = "/content/output"
            self.MODELS_DIR = "/content/models"
            
            # Crea le directory di output
            os.makedirs(self.OUTPUT_DIR, exist_ok=True)
            os.makedirs(self.MODELS_DIR, exist_ok=True)
            
            # In Colab, BASE_DIR verrà impostato dopo il download
            self.BASE_DIR = "/content/placeholder"  # Verrà sovrascritto dopo il download
            
            # Inizializza i percorsi dei file a None per ora
            self.TRAIN_AUDIO_DIR = None
            self.TEST_SOUNDSCAPES_DIR = None
            self.TRAIN_CSV_PATH = None
            self.TAXONOMY_CSV_PATH = None
            self.SAMPLE_SUB_PATH = None
            
        else:  # locale
            # In ambiente locale, i percorsi dipenderanno dalla tua configurazione
            self.BASE_DIR = os.path.abspath(".")
            self.OUTPUT_DIR = os.path.join(self.BASE_DIR, "output")
            self.MODELS_DIR = os.path.join(self.BASE_DIR, "models")
            
            # Crea le directory se non esistono
            os.makedirs(self.OUTPUT_DIR, exist_ok=True)
            os.makedirs(self.MODELS_DIR, exist_ok=True)
            
            # Imposta i percorsi derivati
            self._setup_derived_paths()
        
        # Parametri per il preprocessing audio - adattati per PASST
        self.SR = 32000      # Sample rate
        self.DURATION = 5    # Durata dei clip in secondi
        self.N_MELS = 128    # Numero di bande Mel
        self.N_FFT = 1024    # Dimensione finestra FFT
        self.HOP_LENGTH = 500  # Hop length per STFT
        self.FMIN = 40       # Frequenza minima per lo spettrogramma Mel
        self.FMAX = 15000    # Frequenza massima
        self.POWER = 2       # Esponente per calcolo spettrogramma
            
        # Parametri per il training
        self.BATCH_SIZE = 64  # Dimensione del batch per PASST
        self.EPOCHS = 10     # Numero di epoche per il training
        self.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
        self.NUM_WORKERS = 4  # Lavoratori per il data loading
        self.LEARNING_RATE = 1e-4  # Learning rate più basso per transformer
        self.WEIGHT_DECAY = 1e-5   # Weight decay per la regolarizzazione

        # Parametri specifici per PASST
        self.PATCH_SIZE = 16      # Dimensione dei patch per il transformer
        self.HIDDEN_DIM = 768     # Dimensione dello strato nascosto
        self.NUM_HEADS = 12       # Numero di teste di attention
        self.NUM_LAYERS = 12      # Numero di layer transformer
        self.MLP_RATIO = 4        # Rapporto di espansione per MLP
        self.DROPOUT = 0.1        # Dropout rate

        # Parametri per inference/submission
        self.TEST_CLIP_DURATION = 5  # Durata dei segmenti per la predizione (secondi)
        self.N_CLASSES = 0  # Sarà impostato dopo aver caricato i dati

    def _setup_derived_paths(self):
        """Imposta i percorsi derivati basati su BASE_DIR"""
        # Utilizza la normale divisione di percorso di OS (non il backslash hardcoded)
        self.TRAIN_AUDIO_DIR = os.path.join(self.BASE_DIR, "train_audio")
        self.TEST_SOUNDSCAPES_DIR = os.path.join(self.BASE_DIR, "test_soundscapes")
        self.TRAIN_CSV_PATH = os.path.join(self.BASE_DIR, "train.csv")
        self.TAXONOMY_CSV_PATH = os.path.join(self.BASE_DIR, "taxonomy.csv") 
        self.SAMPLE_SUB_PATH = os.path.join(self.BASE_DIR, "sample_submission.csv")

In [ ]:
config = Config()

# Gestione download dati in Colab con kagglehub
if config.environment == 'colab':
    # Percorsi nella cache di kagglehub
    cache_competition_path = "/root/.cache/kagglehub/competitions/birdclef-2025"
    cache_model_path = "/root/.cache/kagglehub/models/maurocarlu/passt-bird/PyTorch/default/1"
    
    # Verifica se i dati sono già presenti nella cache
    data_exists = os.path.exists(os.path.join(cache_competition_path, "train.csv"))
    model_exists = os.path.exists(os.path.join(cache_model_path, "passt_model.pth"))
    
    if data_exists and model_exists:
        print("I dati e il modello sono già presenti nella cache. Utilizzo copie esistenti.")
        birdclef_path = cache_competition_path
        model_path = cache_model_path
    else:
        print("Scaricamento dati con kagglehub...")
        
        try:
            import kagglehub
            
            # Scarica solo i dati della competizione se necessario
            if not data_exists:
                print("Download dataset...")
                kagglehub.login()  # Mostra dialog di login interattivo
                birdclef_path = kagglehub.competition_download('birdclef-2025')
            else:
                print("Dataset già presente nella cache.")
                birdclef_path = cache_competition_path
                
            # Scarica solo il modello se necessario
            if not model_exists:
                print("Download modello...")
                kagglehub.login()  # Potrebbe essere necessario riautenticarsi
                model_path = kagglehub.model_download('maurocarlu/passt-bird/PyTorch/default/1')
            else:
                print("Modello già presente nella cache.")
                model_path = cache_model_path
                
            print(f"Download completato.")
            
        except Exception as e:
            print(f"Errore durante il download dei dati: {e}")
            print("Prova ad usare Google Drive o esegui su Kaggle.")
            
            # Se il download fallisce ma i dati esistono parzialmente, usa quelli
            if os.path.exists(cache_competition_path):
                birdclef_path = cache_competition_path
                print(f"Usando i dati esistenti in: {birdclef_path}")
            if os.path.exists(cache_model_path):
                model_path = cache_model_path
                print(f"Usando il modello esistente in: {model_path}")
    
    # Aggiorna i percorsi nella configurazione
    config.BASE_DIR = birdclef_path
    config._setup_derived_paths()
    config.MODELS_DIR = model_path
    
    print(f"Dati disponibili in: {config.BASE_DIR}")
    print(f"Modello disponibile in: {config.MODELS_DIR}")

# Stampa percorsi aggiornati
print(f"\nPercorso file CSV di training: {config.TRAIN_CSV_PATH}")
print(f"Percorso directory audio di training: {config.TRAIN_AUDIO_DIR}")

In [ ]:
# Verifica l'esistenza delle directory e crea quelle necessarie per l'output

def setup_output_directories():
    """
    Configura le directory per l'output del progetto.
    
    Returns:
        dict: Dictionary con i percorsi delle directory di output
    """
    # Directory principale di output
    output_dir = config.OUTPUT_DIR
    
    # Sotto-directory per diversi tipi di output
    dirs = {
        'checkpoints': os.path.join(output_dir, 'checkpoints'),
        'tensorboard': os.path.join(output_dir, 'tensorboard_logs'),
        'predictions': os.path.join(output_dir, 'predictions'),
        'submissions': os.path.join(output_dir, 'submissions'),
        'visualizations': os.path.join(output_dir, 'visualizations'),
    }
    
    # Crea tutte le directory
    for dir_name, dir_path in dirs.items():
        os.makedirs(dir_path, exist_ok=True)
        print(f"Directory '{dir_name}' creata/verificata in: {dir_path}")
    
    return dirs

# Configura le directory di output
output_dirs = setup_output_directories()

# Crea un file di log per tenere traccia dei risultati
log_file_path = os.path.join(config.OUTPUT_DIR, f"experiment_log_passt_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.txt")

with open(log_file_path, 'w') as log_file:
    log_file.write(f"=== BirdClef PASST Experiment Log ===\n")
    log_file.write(f"Date: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    log_file.write(f"Environment: {config.environment}\n\n")
    log_file.write("Output directories:\n")
    for dir_name, dir_path in output_dirs.items():
        log_file.write(f"- {dir_name}: {dir_path}\n")

print(f"File di log creato in: {log_file_path}")

# Memorizziamo i parametri di configurazione principali per l'addestramento
print("\nParametri di configurazione principali:")
print(f"- Sample rate: {config.SR} Hz")
print(f"- Durata clip audio: {config.DURATION} secondi")
print(f"- Numero bande Mel: {config.N_MELS}")
print(f"- Dimensione FFT: {config.N_FFT}")
print(f"- Hop length: {config.HOP_LENGTH}")
print(f"- Device: {config.DEVICE}")
print(f"- Batch size: {config.BATCH_SIZE}")
print(f"- Epoche: {config.EPOCHS}")
print(f"- Parametri PASST: {config.HIDDEN_DIM} hidden dim, {config.NUM_HEADS} attention heads, {config.NUM_LAYERS} layers")

In [ ]:
# Caricamento dei metadati
def load_metadata():
    """
    Carica e prepara i metadati dal file CSV di training.
    
    Returns:
        tuple: training_df, all_species, labels_one_hot
    """
    print(f"Caricamento metadati da: {config.TRAIN_CSV_PATH}")
    train_df = pd.read_csv(config.TRAIN_CSV_PATH)
    sample_sub_df = pd.read_csv(config.SAMPLE_SUB_PATH)
    
    # Estrai tutte le etichette uniche
    train_primary_labels = train_df['primary_label'].unique()
    train_secondary_labels = set([lbl for sublist in train_df['secondary_labels'].apply(eval) 
                                 for lbl in sublist if lbl])
    submission_species = sample_sub_df.columns[1:].tolist()  # Escludi row_id
    
    # Combina tutte le possibili etichette
    all_species = sorted(list(set(train_primary_labels) | train_secondary_labels | set(submission_species)))
    N_CLASSES = len(all_species)
    config.N_CLASSES = N_CLASSES  # Aggiorna il numero di classi nella configurazione
    
    print(f"Numero totale di specie trovate: {N_CLASSES}")
    print(f"Prime 10 specie: {all_species[:10]}")
    
    # Crea mappatura etichette-indici
    species_to_int = {species: i for i, species in enumerate(all_species)}
    int_to_species = {i: species for species, i in species_to_int.items()}
    
    # Aggiungi indici numerici al dataframe
    train_df['primary_label_int'] = train_df['primary_label'].map(species_to_int)
    
    # Prepara target multi-etichetta
    mlb = MultiLabelBinarizer(classes=all_species)
    mlb.fit(None)  # Fit con tutte le classi
    
    def get_multilabel(row):
        labels = eval(row['secondary_labels'])  # Valuta la lista di stringhe in modo sicuro
        labels.append(row['primary_label'])
        return list(set(labels))  # Assicura etichette uniche
    
    train_df['all_labels'] = train_df.apply(get_multilabel, axis=1)
    train_labels_one_hot = mlb.transform(train_df['all_labels'])
    
    print(f"Forma delle etichette one-hot: {train_labels_one_hot.shape}")
    
    return train_df, all_species, train_labels_one_hot, species_to_int, int_to_species

# Carica i metadati
train_df, all_species, train_labels_one_hot, species_to_int, int_to_species = load_metadata()

# Suddividi i dati in training e validation
def split_data(train_df, labels_one_hot, test_size=0.2, random_state=42):
    """
    Suddivide il dataset in set di training e validation.
    
    Args:
        train_df: DataFrame con i metadati
        labels_one_hot: Array di etichette one-hot
        test_size: Percentuale dei dati da usare per validation
        random_state: Seed per riproducibilità
        
    Returns:
        tuple: X_train_df, X_val_df, y_train_one_hot, y_val_one_hot
    """
    # Indici per lo split
    train_indices, val_indices = train_test_split(
        range(len(train_df)),
        test_size=test_size,
        random_state=random_state,
        stratify=train_df['primary_label']  # Stratifica in base alla label primaria
    )
    
    # Crea i dataframe e gli array di etichette splittati
    X_train_df = train_df.iloc[train_indices].reset_index(drop=True)
    X_val_df = train_df.iloc[val_indices].reset_index(drop=True)
    
    y_train_one_hot = labels_one_hot[train_indices]
    y_val_one_hot = labels_one_hot[val_indices]
    
    print(f"Dimensioni Training Set: {X_train_df.shape}, Etichette: {y_train_one_hot.shape}")
    print(f"Dimensioni Validation Set: {X_val_df.shape}, Etichette: {y_val_one_hot.shape}")
    
    return X_train_df, X_val_df, y_train_one_hot, y_val_one_hot

# Suddividi i dati in training e validation
X_train_df, X_val_df, y_train_one_hot, y_val_one_hot = split_data(train_df, train_labels_one_hot)

# Test_df sarà None per ora
X_test_df = None
y_test_one_hot = None

In [ ]:
def create_balanced_dataset_df(train_df, labels_one_hot, abundant_class_threshold=200, remove_percentage=0.3, random_state=42):
    """
    Crea un DataFrame bilanciato rimuovendo parte degli esempi con rating bassi dalle classi abbondanti.
    
    Args:
        train_df: DataFrame originale
        labels_one_hot: Array di etichette one-hot
        abundant_class_threshold: Soglia per definire una classe come "abbondante"
        remove_percentage: Percentuale di esempi con rating 1-3 da rimuovere dalle classi abbondanti
        random_state: Seed per riproducibilità
        
    Returns:
        tuple: (DataFrame bilanciato, etichette one-hot bilanciate)
    """
    # Conta esempi per ogni classe
    class_counts = train_df['primary_label'].value_counts()
    
    # Identifica classi abbondanti
    abundant_classes = class_counts[class_counts > abundant_class_threshold].index.tolist()
    print(f"Classi identificate come abbondanti (>{abundant_class_threshold} esempi): {len(abundant_classes)}")
    
    # Copia il DataFrame originale
    balanced_df = train_df.copy()
    rows_to_drop = []
    
    # Contatori per statistiche
    total_removed = 0
    removed_by_class = {}
    
    # Per ogni classe abbondante
    for cls in abundant_classes:
        # Filtra esempi con rating 1-3 per questa classe
        low_quality_mask = (balanced_df['primary_label'] == cls) & (balanced_df['rating'].isin([1, 2, 3]))
        low_quality_indices = balanced_df[low_quality_mask].index.tolist()
        
        # Numero di esempi da rimuovere
        n_to_remove = int(len(low_quality_indices) * remove_percentage)
        
        # Seleziona casualmente gli indici da rimuovere
        np.random.seed(random_state)
        if n_to_remove > 0:
            indices_to_remove = np.random.choice(low_quality_indices, size=n_to_remove, replace=False)
            
            # Memorizza gli indici da rimuovere
            rows_to_drop.extend(indices_to_remove)
            
            # Aggiorna statistiche
            removed_by_class[cls] = n_to_remove
            total_removed += n_to_remove
    
    # Rimuovi le righe selezionate
    if rows_to_drop:
        balanced_df = balanced_df.drop(rows_to_drop).reset_index(drop=True)
        
        # Aggiorna anche le etichette one-hot rimuovendo gli stessi indici
        mask = np.ones(len(train_df), dtype=bool)
        mask[rows_to_drop] = False
        balanced_labels = labels_one_hot[mask]
    else:
        balanced_labels = labels_one_hot
    
    # Statistiche finali
    print(f"Totale esempi rimossi: {total_removed} ({total_removed/len(train_df):.1%} del dataset originale)")
    print(f"Dimensione dataset originale: {len(train_df)}")
    print(f"Dimensione dataset bilanciato: {len(balanced_df)}")
    
    return balanced_df, balanced_labels

In [ ]:
class AudioAugmentations:
    def __init__(self, p_time_shift=0.5, p_time_mask=0.5, p_freq_mask=0.5, p_mixup=0.3):
        """
        Inizializza le trasformazioni per data augmentation audio per il modello PASST.
        
        Args:
            p_time_shift: Probabilità di applicare time shifting
            p_time_mask: Probabilità di applicare mascheramento temporale
            p_freq_mask: Probabilità di applicare mascheramento frequenziale
            p_mixup: Probabilità di applicare mixup
        """
        self.p_time_shift = p_time_shift
        self.p_time_mask = p_time_mask
        self.p_freq_mask = p_freq_mask
        self.p_mixup = p_mixup
        
    def apply_time_shift(self, spec):
        """Applica time shift allo spettrogramma"""
        if torch.rand(1).item() < self.p_time_shift:
            shift_amount = int(spec.shape[2] * 0.2)  # Shift fino al 20% della lunghezza
            direction = 1 if torch.rand(1).item() > 0.5 else -1
            shift = torch.randint(1, shift_amount + 1, (1,)).item() * direction
            spec = torch.roll(spec, shifts=shift, dims=2)
        return spec
        
    def apply_time_mask(self, spec):
        """Applica maschere casuali sull'asse temporale (X) dello spettrogramma"""
        if torch.rand(1).item() < self.p_time_mask:
            _, _, width = spec.shape
            mask_len = int(width * torch.rand(1).item() * 0.2)  # Maschera fino al 20% della larghezza
            mask_start = torch.randint(0, width - mask_len, (1,))
            spec[:, :, mask_start:mask_start+mask_len] = 0
        return spec
        
    def apply_freq_mask(self, spec):
        """Applica maschere casuali sull'asse frequenziale (Y) dello spettrogramma"""
        if torch.rand(1).item() < self.p_freq_mask:
            _, height, _ = spec.shape
            mask_len = int(height * torch.rand(1).item() * 0.2)  # Maschera fino al 20% dell'altezza
            mask_start = torch.randint(0, height - mask_len, (1,))
            spec[:, mask_start:mask_start+mask_len, :] = 0
        return spec
        
    def apply_all(self, spec):
        """Applica tutte le augmentations in cascata"""
        spec = self.apply_time_shift(spec)
        spec = self.apply_time_mask(spec)
        spec = self.apply_freq_mask(spec)
        return spec

# Funzione per fare mixup tra esempi nel batch
def mixup_batch(inputs, targets, alpha=0.4):
    """
    Applica mixup tra esempi di un batch.
    
    Args:
        inputs: Tensor di input [batch_size, channels, height, width]
        targets: Tensor di target [batch_size, num_classes]
        alpha: Parametro per distribuzione beta
        
    Returns:
        tuple: (inputs mixati, targets mixati)
    """
    batch_size = inputs.size(0)
    indices = torch.randperm(batch_size)
    
    # Preleva lambda dalla distribuzione beta
    lam = np.random.beta(alpha, alpha)
    
    # Mixa gli input
    mixed_inputs = lam * inputs + (1 - lam) * inputs[indices]
    
    # Mixa i target
    mixed_targets = lam * targets + (1 - lam) * targets[indices]
    
    return mixed_inputs, mixed_targets

In [ ]:
def load_and_preprocess_audio(file_path, target_sr=config.SR, duration=config.DURATION, segment_position='center', random_segment=False):
    """
    Carica un file audio, estrae un segmento specifico e lo converte in spettrogramma Mel.
    
    Args:
        file_path: Percorso del file audio
        target_sr: Sample rate target
        duration: Durata target in secondi
        segment_position: Posizione del segmento ('start', 'center', 'end')
        random_segment: Se True, estrae un segmento casuale
        
    Returns:
        numpy.ndarray: Spettrogramma Mel log-normalizzato
    """
    try:
        # Carica il file audio
        y, sr = librosa.load(file_path, sr=target_sr, mono=True)
        
        # Lunghezza target in campioni
        target_len = int(target_sr * duration)
        total_len = len(y)
        
        # Gestisci clip troppo corte
        if total_len < target_len:
            import math
            n_copy = math.ceil(target_len / total_len)
            if n_copy > 1:
                y = np.tile(y, n_copy)
            total_len = len(y)
        
        # Seleziona il segmento
        if random_segment and total_len > target_len:
            # Estrai segmento casuale
            max_start_idx = total_len - target_len
            start_idx = np.random.randint(0, max_start_idx)
        else:
            # Usa le posizioni predefinite
            if segment_position == 'start':
                start_idx = int(total_len * 0.2)
                if start_idx + target_len > total_len:
                    start_idx = max(0, total_len - target_len)
            elif segment_position == 'end':
                end_point = int(total_len * 0.8)
                start_idx = max(0, end_point - target_len)
            else:  # 'center' (default)
                start_idx = max(0, int(total_len / 2 - target_len / 2))
        
        # Estrai il segmento
        y = y[start_idx:start_idx + target_len]
        
        # Padda se necessario
        if len(y) < target_len:
            y = np.pad(y, (0, target_len - len(y)), mode='constant')
        
        # Calcola lo spettrogramma Mel
        mel_spec = librosa.feature.melspectrogram(
            y=y, sr=sr,
            n_fft=config.N_FFT,
            hop_length=config.HOP_LENGTH,
            n_mels=config.N_MELS,
            fmin=config.FMIN,
            fmax=config.FMAX,
            power=config.POWER
        )
        
        # Converti in scala logaritmica (dB)
        log_mel_spec = librosa.power_to_db(mel_spec, ref=np.max)
        
        # Normalizza
        min_val = np.min(log_mel_spec)
        max_val = np.max(log_mel_spec)
        if max_val > min_val:
            log_mel_spec = (log_mel_spec - min_val) / (max_val - min_val)
        else:
            log_mel_spec = np.zeros_like(log_mel_spec)
        
        return log_mel_spec
        
    except Exception as e:
        print(f"Errore nell'elaborazione di {file_path}: {e}")
        time_steps = int(target_sr * duration / config.HOP_LENGTH) + 1
        return np.zeros((config.N_MELS, time_steps), dtype=np.float32)

In [ ]:
class BirdDataset(Dataset):
    def __init__(self, df, audio_dir, labels_one_hot, transform=None):
        """
        Dataset che estrae solo il segmento centrale per ogni clip audio.
        """
        self.df = df
        self.audio_dir = audio_dir
        self.labels = labels_one_hot
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        filename = row['filename']
        file_path = os.path.join(self.audio_dir, filename)
        
        if not os.path.exists(file_path):
            print(f"Attenzione: File non trovato in {file_path}.")
            time_steps = int(config.SR * config.DURATION / config.HOP_LENGTH) + 1
            dummy_spec = torch.zeros((1, config.N_MELS, time_steps), dtype=torch.float32)
            dummy_label = torch.zeros(config.N_CLASSES, dtype=torch.float32)
            return dummy_spec, dummy_label
            
        # Carica e preprocessa l'audio con solo segmento centrale
        mel_spec = load_and_preprocess_audio(file_path, segment_position='center')
        
        # Aggiungi dimensione del canale e converti in tensor
        mel_spec = np.expand_dims(mel_spec, axis=0)
        mel_spec_tensor = torch.tensor(mel_spec, dtype=torch.float32)
        
        # Ottieni le etichette
        label_tensor = torch.tensor(self.labels[idx], dtype=torch.float32)
        
        if self.transform:
            mel_spec_tensor = self.transform(mel_spec_tensor)
            
        return mel_spec_tensor, label_tensor

In [ ]:
class AdaptiveBirdDatasetPASST(Dataset):
    def __init__(self, df, audio_dir, labels_one_hot, augmentations=None):
        """
        Dataset adattivo con supporto per augmentation specifiche per PASST.
        
        Args:
            df: DataFrame con i metadati
            audio_dir: Directory contenente i file audio
            labels_one_hot: Array di etichette one-hot
            augmentations: Istanza di AudioAugmentations
        """
        self.df = df
        self.audio_dir = audio_dir
        self.labels = labels_one_hot
        self.augmentations = augmentations
        
        # Pre-calcola quali segmenti usare per ogni clip
        self.segments_to_use = []
        print("Analizzando le lunghezze delle clip audio...")
        
        for idx, row in tqdm(df.iterrows(), total=len(df), desc="Preparazione dataset PASST"):
            file_path = os.path.join(audio_dir, row['filename'])
            try:
                # Carica solo l'informazione sulla durata senza caricare l'intero file
                y, sr = librosa.load(file_path, sr=None, duration=0.1)  # Carica solo un breve segmento per ottenere SR
                info = librosa.get_duration(filename=file_path, sr=sr)
                total_duration = info  # Durata in secondi
                
                # Determina quali segmenti usare in base alla durata
                if total_duration < config.DURATION * 1.5:
                    # Clip troppo corta per multiple segmenti, usa solo il centro
                    segments = ['center']
                elif total_duration < config.DURATION * 2.5:
                    # Clip media, usa inizio e fine
                    segments = ['start', 'end']
                else:
                    # Clip abbastanza lunga, usa tutti e tre i segmenti
                    segments = ['start', 'center', 'end']
                
                # Memorizza l'indice originale e i segmenti da utilizzare
                for segment in segments:
                    self.segments_to_use.append((idx, segment))
                    
            except Exception as e:
                # In caso di errore, usa solo il segmento centrale
                self.segments_to_use.append((idx, 'center'))
                print(f"Errore nell'elaborazione di {file_path}: {e}")
    
    def __len__(self):
        return len(self.segments_to_use)
    
    def __getitem__(self, idx):
        df_idx, segment_position = self.segments_to_use[idx]
        
        # Ottieni il record dal DataFrame originale
        row = self.df.iloc[df_idx]
        filename = row['filename']
        file_path = os.path.join(self.audio_dir, filename)
        
        # Determina se usare un segmento casuale
        use_random_segment = self.augmentations is not None
        
        # Carica e preprocessa l'audio con il segmento selezionato o casuale
        mel_spec = load_and_preprocess_audio(
            file_path, 
            segment_position=segment_position,
            random_segment=use_random_segment
        )
        
        # Aggiungi dimensione del canale e converti in tensor
        mel_spec = np.expand_dims(mel_spec, axis=0)
        mel_spec_tensor = torch.tensor(mel_spec, dtype=torch.float32)
        
        # Applica le augmentations se attive
        if self.augmentations is not None:
            mel_spec_tensor = self.augmentations.apply_all(mel_spec_tensor)
            
        # Ottieni le etichette corrispondenti
        label_tensor = torch.tensor(self.labels[df_idx], dtype=torch.float32)
            
        return mel_spec_tensor, label_tensor

In [ ]:
class PatchEmbed(nn.Module):
    """Convertire le immagini in embedded patches"""
    def __init__(self, img_size=(128, 320), patch_size=16, in_channels=1, embed_dim=768):
        super().__init__()
        self.img_size = img_size
        self.patch_size = patch_size
        self.n_patches = (img_size[0] // patch_size) * (img_size[1] // patch_size)
        
        self.proj = nn.Conv2d(
            in_channels,
            embed_dim,
            kernel_size=patch_size,
            stride=patch_size
        )
        
    def forward(self, x):
        x = self.proj(x)  # (B, embed_dim, patches_H, patches_W)
        x = x.flatten(2)  # (B, embed_dim, n_patches)
        x = x.transpose(1, 2)  # (B, n_patches, embed_dim)
        return x

class Attention(nn.Module):
    def __init__(self, dim, n_heads=12, qkv_bias=True, attn_drop=0., proj_drop=0.):
        super().__init__()
        self.n_heads = n_heads
        self.head_dim = dim // n_heads
        self.scale = self.head_dim ** -0.5
        
        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)
        
    def forward(self, x):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.n_heads, C // self.n_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]
        
        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)
        
        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x

class MLP(nn.Module):
    def __init__(self, in_features, hidden_features, out_features, drop=0.):
        super().__init__()
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = nn.GELU()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x

class Block(nn.Module):
    def __init__(self, dim, n_heads, mlp_ratio=4., qkv_bias=True, drop=0., attn_drop=0.):
        super().__init__()
        self.norm1 = nn.LayerNorm(dim)
        self.attn = Attention(
            dim, n_heads=n_heads, qkv_bias=qkv_bias, attn_drop=attn_drop, proj_drop=drop
        )
        self.norm2 = nn.LayerNorm(dim)
        self.mlp = MLP(
            in_features=dim,
            hidden_features=int(dim * mlp_ratio),
            out_features=dim,
            drop=drop
        )
        
    def forward(self, x):
        x = x + self.attn(self.norm1(x))
        x = x + self.mlp(self.norm2(x))
        return x

class PASSTransformer(nn.Module):
    def __init__(self, img_size=(128, 320), patch_size=16, in_channels=1, 
                 n_classes=config.N_CLASSES, embed_dim=768, depth=12, 
                 n_heads=12, mlp_ratio=4., qkv_bias=True, drop_rate=0.1, attn_drop_rate=0.):
        super().__init__()
        self.n_classes = n_classes
        
        # Patch embedding
        self.patch_embed = PatchEmbed(
            img_size=img_size,
            patch_size=patch_size,
            in_channels=in_channels,
            embed_dim=embed_dim
        )
        
        # Numero di token
        self.n_patches = self.patch_embed.n_patches
        
        # Posizione dei token+CLS
        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(torch.zeros(1, self.n_patches + 1, embed_dim))
        
        self.pos_drop = nn.Dropout(p=drop_rate)
        
        # Blocchi transformer
        self.blocks = nn.ModuleList([
            Block(
                dim=embed_dim, n_heads=n_heads, mlp_ratio=mlp_ratio,
                qkv_bias=qkv_bias, drop=drop_rate, attn_drop=attn_drop_rate
            )
            for _ in range(depth)
        ])
        
        # Normalizzazione finale
        self.norm = nn.LayerNorm(embed_dim)
        
        # Head di classificazione
        self.head = nn.Sequential(
            nn.LayerNorm(embed_dim),
            nn.Linear(embed_dim, n_classes)
        )
        
        # Inizializzazione
        nn.init.normal_(self.pos_embed, std=0.02)
        nn.init.normal_(self.cls_token, std=0.02)
        self.apply(self._init_weights)
        
    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            nn.init.normal_(m.weight, std=0.02)
            if m.bias is not None:
                nn.init.zeros_(m.bias)
        elif isinstance(m, nn.LayerNorm):
            nn.init.zeros_(m.bias)
            nn.init.ones_(m.weight)
            
    def forward_features(self, x):
        B = x.shape[0]
        
        # Embedding dei patch
        x = self.patch_embed(x)
        
        # Aggiungi il token CLS
        cls_tokens = self.cls_token.expand(B, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        
        # Aggiungi l'embedding posizionale
        x = x + self.pos_embed
        x = self.pos_drop(x)
        
        # Applica i blocchi transformer
        for block in self.blocks:
            x = block(x)
            
        # Normalizzazione finale
        x = self.norm(x)
        
        # Restituisci solo il token CLS
        return x[:, 0]
        
    def forward(self, x):
        x = self.forward_features(x)
        x = self.head(x)
        return x

In [ ]:
# Controlla se esiste un checkpoint precedente
has_previous_checkpoint = False
if config.environment == 'kaggle':
    latest_checkpoint = '/kaggle/working/checkpoints/latest_checkpoint.pth'
    has_previous_checkpoint = os.path.exists(latest_checkpoint)
elif config.environment == 'colab':
    drive_checkpoint = '/content/drive/MyDrive/birdclef_checkpoints/latest_checkpoint.pth'
    has_previous_checkpoint = os.path.exists(drive_checkpoint)
else:
    # Per ambienti locali
    local_checkpoint = os.path.join(config.OUTPUT_DIR, 'checkpoints', 'latest_checkpoint.pth')
    has_previous_checkpoint = os.path.exists(local_checkpoint)

# Calcola le dimensioni dell'input
time_steps = int(config.SR * config.DURATION / config.HOP_LENGTH) + 1
img_size = (config.N_MELS, time_steps)
print(f"Dimensione spettrogrammi: {img_size}")

# Inizializza il modello PASST
model = PASSTransformer(
    img_size=img_size,
    patch_size=config.PATCH_SIZE,
    in_channels=1,
    n_classes=config.N_CLASSES,
    embed_dim=config.HIDDEN_DIM,
    depth=config.NUM_LAYERS,
    n_heads=config.NUM_HEADS,
    mlp_ratio=config.MLP_RATIO,
    drop_rate=config.DROPOUT
).to(config.DEVICE)

# Stampa il riepilogo del modello
print(f"Modello PASST creato con {sum(p.numel() for p in model.parameters())/1e6:.1f}M parametri")
print(f"Immagine di input: {img_size}, Patch: {config.PATCH_SIZE}, Canali: 1")
print(f"Dim embed: {config.HIDDEN_DIM}, Heads: {config.NUM_HEADS}, Layers: {config.NUM_LAYERS}")

In [ ]:
# Applica il bilanciamento strategico solo al dataset di training
print("\n=== Bilanciamento Strategico del Dataset di Training ===")
X_train_df_balanced, y_train_one_hot_balanced = create_balanced_dataset_df(
    X_train_df, 
    y_train_one_hot,
    abundant_class_threshold=150,  # Classi con più di 150 esempi sono considerate abbondanti
    remove_percentage=0.4  # Rimuove il 40% degli esempi con rating bassi
)

# Crea un'istanza delle augmentations audio specifiche per PASST
audio_augmentations = AudioAugmentations(
    p_time_shift=0.5, 
    p_time_mask=0.5, 
    p_freq_mask=0.5, 
    p_mixup=0.3
)

# Creiamo i dataset utilizzando il dataset adattivo per il training con augmentations
print("Creazione dataset di training con approccio adattivo e augmentations...")
train_dataset = AdaptiveBirdDatasetPASST(
    X_train_df_balanced, 
    config.TRAIN_AUDIO_DIR, 
    y_train_one_hot_balanced,
    augmentations=audio_augmentations
)

# Per validation, non usiamo augmentations
print("Creazione dataset di validation con segmento centrale...")
val_dataset = BirdDataset(X_val_df, config.TRAIN_AUDIO_DIR, y_val_one_hot)

# Stampa informazioni sulla dimensione effettiva del dataset
print(f"\nNumero di record originali nel training set: {len(X_train_df)}")
print(f"Numero di campioni effettivi nel training set dopo l'adattamento: {len(train_dataset)}")
print(f"Rapporto di espansione: {len(train_dataset) / len(X_train_df):.2f}x")

# Implementa una funzione di collate personalizzata per mixup
def mixup_collate_fn(batch):
    """Collate function con supporto per mixup batch-wise"""
    inputs = []
    targets = []
    
    # Estrai input e target dal batch
    for input_tensor, target_tensor in batch:
        inputs.append(input_tensor)
        targets.append(target_tensor)
    
    # Stack per creare tensor batch
    inputs = torch.stack(inputs)
    targets = torch.stack(targets)
    
    # Applica mixup con 30% di probabilità
    if torch.rand(1).item() < 0.3:
        inputs, targets = mixup_batch(inputs, targets, alpha=0.4)
    
    return inputs, targets

# Creiamo i dataloader con mixup per il training
train_loader = DataLoader(
    train_dataset, 
    batch_size=config.BATCH_SIZE, 
    shuffle=True,
    num_workers=config.NUM_WORKERS, 
    pin_memory=True,
    collate_fn=mixup_collate_fn
)

# Per validation, non usiamo mixup
val_loader = DataLoader(
    val_dataset, 
    batch_size=config.BATCH_SIZE, 
    shuffle=False,
    num_workers=config.NUM_WORKERS, 
    pin_memory=True
)

# Non creiamo un test_loader per ora
test_loader = None
    
print(f"Numero di batch di training per epoca: {len(train_loader)}")
print(f"Numero di batch di validation per epoca: {len(val_loader)}")
print("Test set: utilizzeremo direttamente i file nella cartella test_soundscapes")

# Ottimizzatore con weight decay - stile ViT
optimizer = optim.AdamW(
    model.parameters(),
    lr=config.LEARNING_RATE,
    weight_decay=config.WEIGHT_DECAY,
    betas=(0.9, 0.999)
)

# Learning rate scheduler - CosineAnnealingLR con warmup
def get_cosine_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps, min_lr=1e-6):
    def lr_lambda(current_step):
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        progress = float(current_step - num_warmup_steps) / float(max(1, num_training_steps - num_warmup_steps))
        return max(min_lr, 0.5 * (1.0 + math.cos(math.pi * progress)))
    
    return torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

# Calcola il numero totale di step di training
total_steps = len(train_loader) * config.EPOCHS
warmup_steps = int(total_steps * 0.1)  # 10% di warmup

scheduler = get_cosine_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps,
    min_lr=1e-6
)

# Loss function - BCEWithLogitsLoss per classificazione multi-label
criterion = nn.BCEWithLogitsLoss()

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler,
                epochs=config.EPOCHS, device=config.DEVICE, 
                model_save_path=None, model_load_path=None, patience=3,
                resume_training=True):
    """
    Addestra il modello PASST e valuta su validation set con supporto per checkpoint.
    
    Args:
        model: Modello PyTorch da addestrare
        train_loader: DataLoader per dati di training
        val_loader: DataLoader per dati di validation
        criterion: Funzione di loss
        optimizer: Ottimizzatore
        scheduler: Learning rate scheduler
        epochs: Numero di epoche di training
        device: Device per l'addestramento ('cuda' o 'cpu')
        model_save_path: Path dove salvare il modello addestrato
        model_load_path: Path da cui caricare un modello pre-addestrato
        patience: Numero di epoche senza miglioramento prima di terminare l'addestramento
        resume_training: Se True, riprende il training da un checkpoint (se disponibile)
        
    Returns:
        tuple: (train_losses, val_losses, total_training_time)
    """
    # Directory per i checkpoint in base all'ambiente
    checkpoint_dir = None
    drive_mounted = False
    
    # Configura la directory per i checkpoint a seconda dell'ambiente
    if config.environment == 'colab':
        try:
            from google.colab import drive
            # Controlla se il drive è già montato
            if not os.path.exists('/content/drive'):
                print("Montaggio di Google Drive...")
                drive.mount('/content/drive')
                print("Google Drive montato con successo.")
            
            # Crea directory per i checkpoint se non esiste
            checkpoint_dir = '/content/drive/MyDrive/birdclef_checkpoints'
            os.makedirs(checkpoint_dir, exist_ok=True)
            print(f"Directory per i checkpoint creata su Google Drive: {checkpoint_dir}")
            
            # Aggiorna il percorso di salvataggio per usare Google Drive
            if model_save_path:
                filename = os.path.basename(model_save_path)
                model_save_path = os.path.join(checkpoint_dir, filename)
                print(f"Il modello sarà salvato in: {model_save_path}")
            
            drive_mounted = True
        except ImportError:
            print("Errore: Non riesco ad accedere a Google Drive. Continuo senza persistenza.")
        except Exception as e:
            print(f"Errore durante il montaggio di Google Drive: {e}")
            print("Continuo senza persistenza su Drive.")
    elif config.environment == 'kaggle':
        # In Kaggle, usa la directory di working
        checkpoint_dir = '/kaggle/working/checkpoints'
        os.makedirs(checkpoint_dir, exist_ok=True)
        print(f"Directory per i checkpoint creata in Kaggle: {checkpoint_dir}")
    else:
        # In locale, usa la directory 'checkpoints' nell'OUTPUT_DIR
        checkpoint_dir = os.path.join(config.OUTPUT_DIR, 'checkpoints')
        os.makedirs(checkpoint_dir, exist_ok=True)
        print(f"Directory per i checkpoint creata in locale: {checkpoint_dir}")
    
    # Inizializzazione variabili
    train_losses = []
    val_losses = []
    best_val_loss = float('inf')
    epochs_without_improvement = 0
    total_training_time = 0
    start_epoch = 0
    needs_training = True
    checkpoint_exists = False
    model_loaded = False
    
    # Verifica se esiste un modello pre-addestrato da caricare
    if model_load_path and os.path.exists(model_load_path):
        print(f"Modello trovato in {model_load_path}. Tentativo di caricamento...")
        try:
            checkpoint = torch.load(model_load_path, map_location=device)
            
            if isinstance(checkpoint, dict) and 'model_state_dict' in checkpoint:
                model.load_state_dict(checkpoint['model_state_dict'])
            else:
                model.load_state_dict(checkpoint)
                
            print("Modello caricato con successo.")
            model_loaded = True
            needs_training = False
        except Exception as e:
            print(f"Errore durante il caricamento del modello: {e}")
            print("Verrà avviato l'addestramento da zero.")
            needs_training = True
    else:
        print(f"Modello non trovato in {model_load_path}.")
    
    # Cerca un checkpoint SOLO se il caricamento del modello è fallito E resume_training è True
    if needs_training and resume_training and checkpoint_dir and not model_loaded:
        latest_checkpoint = os.path.join(checkpoint_dir, "latest_checkpoint_passt.pth")
        if os.path.exists(latest_checkpoint):
            print(f"Trovato checkpoint in {latest_checkpoint}. Tentativo di caricamento...")
            try:
                checkpoint = torch.load(latest_checkpoint, map_location=device)
                
                # Verifica che sia un checkpoint compatibile prima di caricarlo
                if isinstance(checkpoint, dict) and 'epoch' in checkpoint:
                    try:
                        model.load_state_dict(checkpoint['model_state_dict'])
                        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
                        start_epoch = checkpoint['epoch'] + 1
                        train_losses = checkpoint['train_losses']
                        val_losses = checkpoint['val_losses']
                        best_val_loss = checkpoint['best_val_loss']
                        epochs_without_improvement = checkpoint['epochs_without_improvement']
                        total_training_time = checkpoint.get('total_training_time', 0)
                        
                        # Ricrea lo scheduler con lo stato salvato se presente
                        if scheduler is not None and 'scheduler_state_dict' in checkpoint:
                            scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
                        
                        print(f"Checkpoint caricato con successo (epoca {start_epoch-1})")
                        print(f"Si riparte dall'epoca {start_epoch}/{epochs}")
                        
                        if start_epoch >= epochs:
                            needs_training = False
                        
                        checkpoint_exists = True
                    except Exception as e:
                        print(f"Il checkpoint non è compatibile con il modello attuale: {e}")
                        print("Verrà avviato l'addestramento da zero.")
            except Exception as e:
                print(f"Errore durante il caricamento del checkpoint: {e}")
                print("Si procederà con il training da zero.")
    
    model.to(device)
    
    # Esegui training solo se necessario
    if needs_training:
        start_time_total = time.time()
        model.train()
        
        # Loop di training sulle epoche (inizia da start_epoch)
        for epoch in range(start_epoch, epochs):
            epoch_start_time = time.time()
            
            # --- Fase di Training ---
            model.train()
            running_loss = 0.0
            pbar_train = tqdm(enumerate(train_loader), total=len(train_loader), 
                             desc=f"Epoca {epoch+1}/{epochs} [Train]", leave=True)
            
            for i, (inputs, labels) in pbar_train:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                
                # Aggiorna lo scheduler ad ogni step
                if scheduler is not None:
                    scheduler.step()
                
                running_loss += loss.item()
                avg_loss = running_loss / (i + 1)
                pbar_train.set_postfix({'loss': f"{avg_loss:.4f}"})
            
            epoch_train_loss = running_loss / len(train_loader)
            train_losses.append(epoch_train_loss)
            
            # --- Fase di Validation ---
            model.eval()
            running_val_loss = 0.0
            pbar_val = tqdm(enumerate(val_loader), total=len(val_loader), 
                           desc=f"Epoca {epoch+1}/{epochs} [Val]", leave=True)
            
            with torch.no_grad():
                for i, (val_inputs, val_labels) in pbar_val:
                    val_inputs = val_inputs.to(device)
                    val_labels = val_labels.to(device)
                    
                    val_outputs = model(val_inputs)
                    val_loss = criterion(val_outputs, val_labels)
                    running_val_loss += val_loss.item()
                    avg_val_loss = running_val_loss / (i + 1)
                    pbar_val.set_postfix({'val_loss': f"{avg_val_loss:.4f}"})
            
            epoch_val_loss = running_val_loss / len(val_loader)
            val_losses.append(epoch_val_loss)
            
            epoch_end_time = time.time()
            epoch_duration = epoch_end_time - epoch_start_time
            total_training_time += epoch_duration
            
            print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {epoch_train_loss:.4f}, "
                  f"Val Loss: {epoch_val_loss:.4f}, Duration: {epoch_duration:.2f} sec")
            
            # Salvataggio checkpoint per ogni epoca (in qualsiasi ambiente)
            if checkpoint_dir:
                checkpoint_path = os.path.join(checkpoint_dir, f"passt_epoch_{epoch+1}.pth")
                
                # Salva checkpoint completo con tutte le informazioni di stato
                checkpoint = {
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'train_losses': train_losses,
                    'val_losses': val_losses,
                    'best_val_loss': best_val_loss,
                    'epochs_without_improvement': epochs_without_improvement,
                    'total_training_time': total_training_time
                }
                
                # Salva anche lo stato dello scheduler se esiste
                if scheduler is not None:
                    checkpoint['scheduler_state_dict'] = scheduler.state_dict()
                
                torch.save(checkpoint, checkpoint_path)
                print(f"Checkpoint completo salvato in {checkpoint_path}")
                
                # Aggiorna anche il checkpoint più recente (sovrascrive)
                torch.save(checkpoint, os.path.join(checkpoint_dir, "latest_checkpoint_passt.pth"))
            
            # Early stopping
            if epoch_val_loss < best_val_loss:
                best_val_loss = epoch_val_loss
                epochs_without_improvement = 0
                # Salva il miglior modello separatamente
                if model_save_path:
                    best_path = model_save_path.replace('.pth', '_best.pth')
                    
                    # Salva checkpoint completo
                    best_checkpoint = {
                        'epoch': epoch,
                        'model_state_dict': model.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                        'train_losses': train_losses,
                        'val_losses': val_losses,
                        'best_val_loss': best_val_loss
                    }
                    
                    # Salva anche lo stato dello scheduler
                    if scheduler is not None:
                        best_checkpoint['scheduler_state_dict'] = scheduler.state_dict()
                    
                    torch.save(best_checkpoint, best_path)
                    print(f"Salvato miglior modello in {best_path}")
            else:
                epochs_without_improvement += 1
                
            if epochs_without_improvement >= patience:
                print(f"\nEarly stopping attivato! Nessun miglioramento per {patience} epoche consecutive.")
                break
        
        end_time_total = time.time()
        if checkpoint_exists:
            total_training_time += (end_time_total - start_time_total)
        else:
            total_training_time = end_time_total - start_time_total
            
        print(f"\nTraining terminato in {total_training_time/60:.2f} minuti totali")
        
        # Salva il modello finale
        if model_save_path:
            final_checkpoint = {
                'epoch': epochs-1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_losses': train_losses,
                'val_losses': val_losses,
                'best_val_loss': best_val_loss,
                'total_training_time': total_training_time
            }
            
            # Salva anche lo stato dello scheduler
            if scheduler is not None:
                final_checkpoint['scheduler_state_dict'] = scheduler.state_dict()
                
            torch.save(final_checkpoint, model_save_path)
            print(f"Modello finale salvato in {model_save_path}")
    else:
        print("Training non necessario: modello già caricato o training ripreso e completato.")
    
    # Visualizza le curve di loss
    if train_losses and val_losses:
        plt.figure(figsize=(10, 5))
        plt.plot(range(1, len(train_losses) + 1), train_losses, label='Training Loss')
        plt.plot(range(1, len(val_losses) + 1), val_losses, label='Validation Loss')
        plt.xlabel('Epoche')
        plt.ylabel('Loss')
        plt.title('Curve di Loss di Training e Validation')
        plt.legend()
        plt.grid(True)
        plt.show()
        
        # Salva il grafico
        if checkpoint_dir:
            plt_path = os.path.join(checkpoint_dir, 'loss_curves_passt.png')
            plt.savefig(plt_path)
            print(f"Grafico delle curve di loss salvato in {plt_path}")
    
    return train_losses, val_losses, total_training_time

In [ ]:
# Percorsi per caricamento/salvataggio del modello
if config.environment == 'kaggle':
    # Directory per i checkpoint in Kaggle
    os.makedirs('/kaggle/working/checkpoints', exist_ok=True)
    
    # Verifica se esiste un checkpoint precedente
    latest_checkpoint = '/kaggle/working/checkpoints/latest_checkpoint_passt.pth'
    if os.path.exists(latest_checkpoint):
        model_load_path = latest_checkpoint
        print(f"Trovato checkpoint precedente in {latest_checkpoint}")
    else:
        # Usa un modello base precaricato se disponibile
        model_load_path = "/kaggle/input/passt-model/passt_pretrained.pth"
        
    # Imposta il percorso di salvataggio
    model_save_path = "/kaggle/working/birdclef_trained_model_passt.pth"
    
elif config.environment == 'colab':
    # Per Colab, verifica se esiste un checkpoint su Drive
    drive_checkpoint = '/content/drive/MyDrive/birdclef_checkpoints/latest_checkpoint_passt.pth'
    if os.path.exists(drive_checkpoint):
        model_load_path = drive_checkpoint
        print(f"Trovato checkpoint precedente su Drive: {drive_checkpoint}")
    else:
        # Usa un modello preaddestrato se disponibile
        model_load_path = os.path.join(config.MODELS_DIR, "passt_model.pth") if os.path.exists(os.path.join(config.MODELS_DIR, "passt_model.pth")) else None
    
    model_save_path = os.path.join(config.OUTPUT_DIR, f"birdclef_model_passt_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.pth")
    
else:
    # Per ambienti locali
    local_checkpoint = os.path.join(config.OUTPUT_DIR, 'checkpoints', 'latest_checkpoint_passt.pth')
    if os.path.exists(local_checkpoint):
        model_load_path = local_checkpoint
        print(f"Trovato checkpoint precedente: {local_checkpoint}")
    else:
        model_load_path = os.path.join(config.MODELS_DIR, "passt_model.pth") if os.path.exists(os.path.join(config.MODELS_DIR, "passt_model.pth")) else None
    
    model_save_path = os.path.join(output_dirs['checkpoints'], f"birdclef_model_passt_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.pth")

# Esegui l'addestramento
print("\n=== Avvio dell'addestramento del modello PASST ===")
train_losses, val_losses, training_time = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,  # Passa lo scheduler
    epochs=config.EPOCHS,
    device=config.DEVICE,
    model_save_path=model_save_path,
    model_load_path=model_load_path,
    resume_training=True
)

print(f"\nAddestramento completato in {training_time/60:.2f} minuti.")

In [ ]:
# Funzione di valutazione del modello
def evaluate_model(model, val_loader, criterion, device=config.DEVICE):
    """
    Valuta il modello sul set di validation.
    
    Args:
        model: Modello da valutare
        val_loader: DataLoader per validation
        criterion: Funzione di loss
        device: Device per l'inferenza
    
    Returns:
        tuple: (val_loss, top1_acc, top3_acc, top5_acc, mAP)
    """
    model.eval()
    val_loss = 0.0
    all_targets = []
    all_predictions = []
    
    with torch.no_grad():
        for inputs, targets in tqdm(val_loader, desc="Validazione"):
            inputs = inputs.to(device)
            targets = targets.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item()
            
            # Converti output in probabilità
            predictions = torch.sigmoid(outputs)
            
            # Salva targets e predictions per il calcolo delle metriche
            all_targets.append(targets.cpu().numpy())
            all_predictions.append(predictions.cpu().numpy())
    
    # Calcola la loss media
    val_loss /= len(val_loader)
    
    # Concatena i risultati
    all_targets = np.concatenate(all_targets)
    all_predictions = np.concatenate(all_predictions)
    
    # Calcola metriche
    from sklearn.metrics import average_precision_score, accuracy_score
    
    # Funzione per calcolare l'accuratezza top-k
    def top_k_accuracy(y_true, y_pred, k):
        # Per ogni esempio, trova l'indice delle k classi con score più alto
        top_k_indices = np.argsort(y_pred, axis=1)[:, -k:]
        # Crea una maschera per le top-k predizioni
        top_k_mask = np.zeros_like(y_pred)
        for i, indices in enumerate(top_k_indices):
            top_k_mask[i, indices] = 1
        # Un esempio è corretto se almeno una delle top-k classi è positiva
        correct = ((top_k_mask * y_true) > 0).sum(axis=1) > 0
        return correct.mean()
    
    # Calcola le metriche
    top1_acc = top_k_accuracy(all_targets, all_predictions, 1)
    top3_acc = top_k_accuracy(all_targets, all_predictions, 3)
    top5_acc = top_k_accuracy(all_targets, all_predictions, 5)
    
    # Mean Average Precision
    mAP = average_precision_score(all_targets, all_predictions, average='macro')
    
    print(f"Validation Loss: {val_loss:.4f}")
    print(f"Top-1 Accuracy: {top1_acc:.4f}")
    print(f"Top-3 Accuracy: {top3_acc:.4f}")
    print(f"Top-5 Accuracy: {top5_acc:.4f}")
    print(f"Mean Average Precision: {mAP:.4f}")
    
    return val_loss, top1_acc, top3_acc, top5_acc, mAP

# Valuta il modello addestrato
print("\n=== Valutazione del Modello PASST ===")
val_metrics = evaluate_model(model, val_loader, criterion)

In [ ]:
def generate_submission(model, device=config.DEVICE):
    """
    Genera un file di submission per Kaggle.
    
    Args:
        model: Modello addestrato
        device: Device per inferenza
        
    Returns:
        pd.DataFrame: DataFrame di submission
    """
    model.eval()
    
    # Set seed per riproducibilità
    np.random.seed(42)
    
    # Percorso dei test soundscapes
    test_soundscape_path = config.TEST_SOUNDSCAPES_DIR
    test_soundscapes = [os.path.join(test_soundscape_path, afile) 
                        for afile in sorted(os.listdir(test_soundscape_path)) 
                        if afile.endswith('.ogg')]
    
    print(f"Elaborazione di {len(test_soundscapes)} file soundscape...")
    
    # Crea DataFrame per le predizioni
    predictions = pd.DataFrame(columns=['row_id'] + all_species)
    
    for soundscape in tqdm(test_soundscapes, desc="Elaborazione soundscapes"):
        # Carica audio
        sig, rate = librosa.load(path=soundscape, sr=config.SR)
        
        # Split in segmenti da 5 secondi
        segment_length = rate * config.TEST_CLIP_DURATION
        chunks = []
        for i in range(0, len(sig), segment_length):
            chunk = sig[i:i+segment_length]
            # Padda se necessario
            if len(chunk) < segment_length:
                chunk = np.pad(chunk, (0, segment_length - len(chunk)), mode='constant')
            chunks.append(chunk)
        
        # Genera predizioni per ogni segmento
        for i, chunk in enumerate(chunks):
            # Calcola row_id (nome file + tempo finale del segmento in secondi)
            file_name = os.path.basename(soundscape).split('.')[0]
            row_id = f"{file_name}_{i * config.TEST_CLIP_DURATION + config.TEST_CLIP_DURATION}"
            
            # Calcola spettrogramma Mel
            mel_spec = librosa.feature.melspectrogram(
                y=chunk, sr=config.SR,
                n_fft=config.N_FFT,
                hop_length=config.HOP_LENGTH,
                n_mels=config.N_MELS,
                fmin=config.FMIN,
                fmax=config.FMAX
            )
            
            # Converti in scala logaritmica (dB) e normalizza
            log_mel_spec = librosa.power_to_db(mel_spec, ref=np.max)
            min_val = np.min(log_mel_spec)
            max_val = np.max(log_mel_spec)
            if max_val > min_val:
                log_mel_spec = (log_mel_spec - min_val) / (max_val - min_val)
            else:
                log_mel_spec = np.zeros_like(log_mel_spec)
            
            # Aggiungi dimensione batch e canale
            log_mel_spec = np.expand_dims(np.expand_dims(log_mel_spec, axis=0), axis=0)
            
            # Converti in tensor
            input_tensor = torch.tensor(log_mel_spec, dtype=torch.float32).to(device)
            
            # Effettua predizione
            with torch.no_grad():
                output = model(input_tensor)
                scores = torch.sigmoid(output).cpu().numpy()[0]
            
            # Aggiungi riga al DataFrame di predizioni
            new_row = pd.DataFrame([[row_id] + list(scores)], columns=['row_id'] + all_species)
            predictions = pd.concat([predictions, new_row], axis=0, ignore_index=True)
    
    # Salva la submission come CSV
    submission_path = os.path.join(config.OUTPUT_DIR, "submission_passt.csv")
    predictions.to_csv(submission_path, index=False)
    print(f"Submission salvata in {submission_path}")
    
    return predictions

# Genera submission solo se siamo su Kaggle
if config.environment == 'kaggle':
    print("\n=== Generazione del File di Submission ===")
    submission_df = generate_submission(model)
    
    if submission_df is not None:
        print("\nAnteprima del file di submission:")
        print(submission_df.head())
else:
    print("\nSalto la generazione della submission perché non siamo su Kaggle.")

In [ ]:
# Esempi di visualizzazione delle attivazioni di attention
def visualize_attention_maps(model, val_loader, device=config.DEVICE, num_examples=3):
    """
    Visualizza le mappe di attenzione del modello PASST su esempi di validation.
    
    Args:
        model: Modello PASST
        val_loader: DataLoader di validation
        device: Dispositivo di esecuzione
        num_examples: Numero di esempi da visualizzare
    """
    model.eval()
    
    # Ottieni alcuni esempi dal validation set
    examples = []
    with torch.no_grad():
        for inputs, targets in val_loader:
            examples.append((inputs, targets))
            if len(examples) >= num_examples:
                break
    
    fig, axes = plt.subplots(num_examples, 3, figsize=(15, 5 * num_examples))
    
    for i, (inputs, targets) in enumerate(examples):
        # Seleziona un solo esempio dal batch
        input_mel = inputs[0]  # [1, H, W]
        target = targets[0]    # [num_classes]
        
        # Visualizza lo spettrogramma originale
        axes[i, 0].imshow(input_mel[0].cpu().numpy(), aspect='auto', origin='lower')
        axes[i, 0].set_title("Spettrogramma Mel")
        axes[i, 0].set_ylabel("Mel Bins")
        axes[i, 0].set_xlabel("Frames")
        
        # Ottieni la classificazione
        model_input = input_mel.unsqueeze(0).to(device)  # [1, 1, H, W]
        with torch.no_grad():
            output = model(model_input)
            probs = torch.sigmoid(output)[0].cpu().numpy()
        
        # Trova le classi con probabilità più alta
        top_indices = np.argsort(probs)[-5:][::-1]
        top_species = [all_species[idx] for idx in top_indices]
        top_probs = [probs[idx] for idx in top_indices]
        
        # Visualizza le probabilità predette
        axes[i, 1].barh(range(5), top_probs)
        axes[i, 1].set_yticks(range(5))
        axes[i, 1].set_yticklabels(top_species)
        axes[i, 1].set_title("Top-5 Predizioni")
        axes[i, 1].set_xlim(0, 1)
        
        # Calcola le classi vere
        true_classes = []
        for j, val in enumerate(target.cpu().numpy()):
            if val > 0:
                true_classes.append(all_species[j])
        
        # Visualizza un'approssimazione dell'attention map (dal token CLS alle posizioni)
        # Nota: questo è solo un esempio, in un modello reale dovremmo estrarre l'attention
        # La simuliamo prendendo l'attivazione delle features
        features = model.forward_features(model_input).cpu().numpy()[0]
        
        # Simuliamo una mappa di attenzione ridimensionandola alle dimensioni dello spettrogramma
        # In un'implementazione reale, accederemmo alle vere mappe di attention del transformer
        attention_map = np.ones((config.N_MELS, time_steps))
        
        # Visualizza l'attention map simulata
        axes[i, 2].imshow(attention_map, aspect='auto', origin='lower', cmap='viridis')
        axes[i, 2].set_title(f"Classi vere: {', '.join(true_classes)}")
        axes[i, 2].set_ylabel("Mel Bins")
        axes[i, 2].set_xlabel("Frames")
    
    plt.tight_layout()
    plt.show()

    # Salva la figura
    if config.environment != 'local':
        fig_path = os.path.join(config.OUTPUT_DIR, 'passt_visualizations.png')
        plt.savefig(fig_path)
        print(f"Visualizzazioni salvate in {fig_path}")

# Visualizza alcuni esempi se non siamo in ambiente Kaggle
if config.environment != 'kaggle':
    print("\n=== Visualizzazione di Esempi ===")
    try:
        visualize_attention_maps(model, val_loader, num_examples=3)
    except Exception as e:
        print(f"Errore durante la visualizzazione: {e}")

In [ ]:
# Confronto tra i modelli EfficientNet e PASST (se disponibili)
try:
    # Tenta di caricare un modello EfficientNet (se è stato addestrato)
    efficientnet_path = os.path.join(config.OUTPUT_DIR, 'checkpoints', 'birdclef_model_efficientnet_best.pth')
    
    if os.path.exists(efficientnet_path):
        print("\n=== Confronto tra EfficientNet e PASST ===")
        print("Caricamento del modello EfficientNet per confronto...")
        
        # Qui si potrebbe implementare il caricamento del modello EfficientNet
        # e confrontare le performance con PASST
        
        print("Confronto completato.")
    else:
        print("\nModello EfficientNet non trovato. Il confronto non verrà effettuato.")
    
except Exception as e:
    print(f"Errore nel tentativo di confronto tra modelli: {e}")

print("\n=== Progetto PASST per BirdClef Completato ===")